<a href="https://colab.research.google.com/github/saeedzou/DeepLearning1401-01/blob/main/Assignment%204/q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import gc
import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
!pip install -q transformers
from transformers import BertTokenizer
from transformers import BertModel
!pip install -q hazm
import hazm
import os
device = "cuda" if torch.cuda.is_available() else "cpu"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!git clone https://github.com/amnghd/Persian_poems_corpus.git

Cloning into 'Persian_poems_corpus'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 45.21 MiB | 15.76 MiB/s, done.
Resolving deltas: 100% (3/3), done.


I selected 10 poets with around 10-15k number of beyts so that the combined dataset is almost balanaced.

In [18]:
poets_dict = {}
for poet in os.listdir('Persian_poems_corpus/normalized'):
  with open(f'Persian_poems_corpus/normalized/{poet}', 'r', encoding='utf-8') as f:
      poem = [line.strip() for line in f.readlines()]
      poem = [verse for verse in poem if len(verse)>0]
  poets_dict[poet.split('_')[0]] = len(poem)
selected_poets = {poet:num_mesras for poet, num_mesras in poets_dict.items() if num_mesras>20000 and num_mesras<31000}

In [44]:
data = []
for poet in selected_poets.keys():
  df = pd.read_csv(os.path.join('Persian_poems_corpus/normalized', poet + '_norm.txt'), header=None, names=['text'])
  if len(df) % 2 == 1:
    df = df[:-1]
  df = pd.DataFrame({'text': [df.iloc[i]['text'] + ' [SEP] ' + df.iloc[i+1]['text'] for i in range(0, len(df), 2)]})
  print(poet, ", number of beyts: ", len(df))
  data.append(df)
result = pd.concat([df.assign(index=i) for i, df in enumerate(data)], axis=0, ignore_index=True)
result = result.sample(frac=1).reset_index(drop=True)

anvari , number of beyts:  13307
farrokhi , number of beyts:  11985
vahshi , number of beyts:  10494
naserkhosro , number of beyts:  10748
seyf , number of beyts:  13008
nezari , number of beyts:  10229
salman , number of beyts:  13015
kamal , number of beyts:  15177
ouhadi , number of beyts:  14620
shahnematollah , number of beyts:  14957


In [45]:
# split data into train and val and test
trainset = result.iloc[:int(len(result)*0.8)].values
valset = result.iloc[int(len(result)*0.8):int(len(result)*0.9)].values
testset = result.iloc[int(len(result)*0.9):].values
# print the length of each dataset
print('train: ', len(trainset))
print('val: ', len(valset))
print('test: ', len(testset))

train:  102032
val:  12754
test:  12754


In [49]:
# calculate max length of poems
max_len = 0
for poem in trainset:
    if len(poem[0].split()) > max_len:
        max_len = len(poem[0].split())
print('max length of poems: ', max_len)
# print the poem with max length
print('poem with max length: ', trainset[np.argmax([len(poem[0].split()) for poem in trainset])][0])

max length of poems:  28
poem with max length:  به وقت صید و سبق و عزم و رزم و از وی فرو ماند  [SEP] به سرعت و هم و جستن برق و رفتن سیل و تک صرصر 


In [50]:
# define a dataset class
class PoemDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.texts = [tokenizer(text, padding='max_length', truncation=True, max_length=max_len, return_tensors='pt') for text in self.data[:, 0]]
        self.labels = [poet for poet in self.data[:, 1]]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.texts[index], torch.tensor(self.labels[index]).long()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')
train_dataset = PoemDataset(trainset, tokenizer, max_len)
val_dataset = PoemDataset(valset, tokenizer, max_len)
test_dataset = PoemDataset(testset, tokenizer, max_len)

In [ ]:
# dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
ParsBERT = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')
# freeze ParsBERT parameters
for param in ParsBERT.parameters():
    param.requires_grad = False
# define a model class
class PoemClassifier(nn.Module):
    def __init__(self, ParsBERT, num_classes):
        super(PoemClassifier, self).__init__()
        self.ParsBERT = ParsBERT
        self.classifier = nn.Linear(768, num_classes)
    
    def forward(self, input_ids, attention_mask):
        output = self.ParsBERT(input_ids=input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        return output

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# define a function for training
def train(model, train_loader, val_loader, optimizer, criterion, epochs, device):
    model = model.to(device)
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_acc = 0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(input_ids=inputs['input_ids'].squeeze(1).to(device), attention_mask=inputs['attention_mask'].to(device))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_acc += (outputs.argmax(1) == labels).sum().item()
        train_loss /= len(train_loader)
        train_acc /= len(train_loader.dataset)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        print(f'Epoch: {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    return train_loss, train_acc, val_loss, val_acc


# define a function for evaluating
def evaluate(model, val_loader, criterion, device):
    model = model.to(device)
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(input_ids=inputs['input_ids'].squeeze(1).to(device), attention_mask=inputs['attention_mask'].to(device))
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_acc += (outputs.argmax(1) == labels).sum().item()
    val_loss /= len(val_loader)
    val_acc /= len(val_loader.dataset)
    return val_loss, val_acc



In [ ]:
# train the model
model = PoemClassifier(ParsBERT, len(poets))
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()
train_loss, train_acc, val_loss, val_acc = train(model, train_loader, val_loader, optimizer, criterion, 5, device)

Epoch: 1/5, Train Loss: 1.8693, Train Acc: 0.3835, Val Loss: 1.8256, Val Acc: 0.3864
Epoch: 2/5, Train Loss: 1.8239, Train Acc: 0.3863, Val Loss: 1.7951, Val Acc: 0.3895
Epoch: 3/5, Train Loss: 1.8053, Train Acc: 0.3885, Val Loss: 1.7799, Val Acc: 0.3931
Epoch: 4/5, Train Loss: 1.7942, Train Acc: 0.3905, Val Loss: 1.7661, Val Acc: 0.3954
Epoch: 5/5, Train Loss: 1.7850, Train Acc: 0.3929, Val Loss: 1.7602, Val Acc: 0.3952


In [ ]:
# save the model
torch.save(model.state_dict(), 'poem_classifier.pt')
# test the model
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

Test Loss: 1.7666, Test Acc: 0.3928


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# define a function for predicting
def predict(model, poem, tokenizer, max_len, device):
    model = model.to(device)
    model.eval()
    poem = tokenizer(poem, padding='max_length', truncation=True, max_length=max_len, return_tensors='pt')
    poem = poem.to(device)
    output = model(input_ids=poem['input_ids'].squeeze(1), attention_mask=poem['attention_mask'])
    return output.argmax(1).item()

# report the results, classification report and confusion matrix
y_true = []
y_pred = []
for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(input_ids=inputs['input_ids'].squeeze(1).to(device), attention_mask=inputs['attention_mask'].to(device))
    y_true.extend(labels.tolist())
    y_pred.extend(outputs.argmax(1).tolist())
print(classification_report(y_true, y_pred, target_names=poets))
print(confusion_matrix(y_true, y_pred))
print(f1_score(y_true, y_pred, average='macro'))

                      precision    recall  f1-score   support

     parvin_norm.txt       0.00      0.00      0.00       570
   shahriar_norm.txt       0.00      0.00      0.00       185
      attar_norm.txt       0.40      0.98      0.57      9517
   farrokhi_norm.txt       0.22      0.01      0.02      1212
      saadi_norm.txt       0.24      0.01      0.01      1565
      bahar_norm.txt       0.28      0.02      0.04      2035
       jami_norm.txt       0.35      0.07      0.12      3274
     sanaee_norm.txt       0.28      0.01      0.03      2647
    moulavi_norm.txt       0.22      0.01      0.01      2612
naserkhosro_norm.txt       0.38      0.01      0.01      1128

            accuracy                           0.39     24745
           macro avg       0.24      0.11      0.08     24745
        weighted avg       0.32      0.39      0.24     24745

[[   0    0  536    2    0    4   23    2    1    2]
 [   0    0  166    2    4    2    6    2    3    0]
 [   0    0 9362    5  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Fine-tuning with SGD and Adam

In [ ]:
# define another model where ParsBERT is not frozen
ParsBERT = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model = PoemClassifier(ParsBERT, len(poets))
# train the model once with SGD optimizer and once with Adam optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# we want to measure perplexity before and after training. select 1000 unbiased samples from the test set
# Get unique labels
labels = np.unique(test[:, 1])
# Initialize empty list to store samples
samples = []
# Loop over unique labels
for label in labels:
    # Get indices of samples with the current label
    indices = np.where(test[:, 1] == label)[0]
    # Randomly select 100 samples from those indices and get the corresponding samples
    samples.append(test[np.random.choice(indices, 100, replace=False)])
# Concatenate all samples into a single numpy array
samples_dataset = PoemDataset(np.concatenate(samples))
# Create a dataloader for the samples
samples_loader = DataLoader(samples_dataset, batch_size=32, shuffle=True)
# test the model
test_loss, test_acc = evaluate(model, samples_loader, criterion, device)
# calculate perplexity
perplexity = np.exp(test_loss)
print(f'Perplexity before training: {perplexity:.4f}')
# train the model once with SGD optimizer
train_loss, train_acc, val_loss, val_acc = train(model, train_loader, val_loader, optimizer, criterion, 10, device)
# save the model
torch.save(model.state_dict(), 'poem_classifier_sgd.pt')
# test the model
test_loss, test_acc = evaluate(model, samples_loader, criterion, device)
# calculate perplexity
perplexity = np.exp(test_loss)
print(f'Perplexity after training: {perplexity:.4f}')
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
# report the results, classification report and confusion matrix
y_true = []
y_pred = []
for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(input_ids=inputs['input_ids'].squeeze(1).to(device), attention_mask=inputs['attention_mask'].to(device))
    y_true.extend(labels.tolist())
    y_pred.extend(outputs.argmax(1).tolist())
print(classification_report(y_true, y_pred, target_names=poets))
print(confusion_matrix(y_true, y_pred))

In [ ]:
# define another model where ParsBERT is not frozen
ParsBERT = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model = PoemClassifier(ParsBERT, len(poets))
# train the model once with SGD optimizer and once with Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# we want to measure perplexity before and after training. select 1000 unbiased samples from the test set
# test the model
test_loss, test_acc = evaluate(model, samples_loader, criterion, device)
# calculate perplexity
perplexity = np.exp(test_loss)
print(f'Perplexity before training: {perplexity:.4f}')
# train the model once with Adam optimizer
train_loss, train_acc, val_loss, val_acc = train(model, train_loader, val_loader, optimizer, criterion, 10, device)
# save the model
torch.save(model.state_dict(), 'poem_classifier_adam.pt')
# test the model
test_loss, test_acc = evaluate(model, samples_loader, criterion, device)
# calculate perplexity
perplexity = np.exp(test_loss)
print(f'Perplexity after training: {perplexity:.4f}')
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
# report the results, classification report and confusion matrix
y_true = []
y_pred = []
for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(input_ids=inputs['input_ids'].squeeze(1).to(device), attention_mask=inputs['attention_mask'].to(device))
    y_true.extend(labels.tolist())
    y_pred.extend(outputs.argmax(1).tolist())
print(classification_report(y_true, y_pred, target_names=poets))
print(confusion_matrix(y_true, y_pred))